## Defining and plotting triangulated surfaces with Plotly `Mesh3d` ##

A triangulation of a compact surface is a finite collection of triangles that cover the surface in  such a way that every point on the surface is in a triangle,  and  the intersection of any two triangles is either void, a common edge or a common vertex. A triangulated surface is called tri-surface.


The triangulation of a surface defined as the graph of a continuous function, $z=f(x,y), (x,y)\in D\subset\mathbb{R}^2$ or in a parametric form:
$$x=x(u,v), y=y(u,v), z=z(u,v), (u,v)\in U\subset\mathbb{R}^2,$$
is the image through $f$, respectively through the parameterization, of the Delaunay triangulation or an user defined triangulation of the planar domain $D$, respectively $U$.

The Delaunay triangulation of a planar region is defined and illustrated in this [Jupyter Notebook](http://nbviewer.jupyter.org/github/empet/Plotly-plots/blob/master/Plotly-Mesh3d.ipynb).

If the planar region $D$  ($U$) is rectangular, then one defines a meshgrid on it, and the points
of the grid are the input  points for the `scipy.spatial.Delaunay` function that defines the  triangulation of $D$, respectively $U$.


### Triangulation of the Moebius band ###

The Moebius band is parameterized by:

$$\begin{align*}
x(u,v)&=(1+0.5 v\cos(u/2))\cos(u)\\
y(u,v)&=(1+0.5 v\cos(u/2))\sin(u)\quad\quad u\in[0,2\pi],\: v\in[-1,1]\\
z(u,v)&=0.5 v\sin(u/2) 
\end{align*}
$$

Define a meshgrid on the rectangle $U=[0,2\pi]\times[-1,1]$:

In [30]:
import numpy as np
import matplotlib.cm as cm
from scipy.spatial import Delaunay

In [64]:
u=np.linspace(0,2*np.pi, 24)
v=np.linspace(-1,1, 8)
u,v=np.meshgrid(u,v)
u=u.flatten()
v=v.flatten()

#evaluate the parameterization at the flattened u and v
tp=1+0.5*v*np.cos(u/2.)
x=tp*np.cos(u)
y=tp*np.sin(u)
z=0.5*v*np.sin(u/2.)

#define 2D points, as input data for the Delaunay triangulation of U
points2D=np.vstack([u,v]).T
tri = Delaunay(points2D)#triangulate the rectangle U

`tri.simplices` is a `np.array` of integers, of shape (`ntri`,3), where `ntri` is the number of triangles generated  by `scipy.spatial.Delaunay`.
Each row in this array contains  three indices,  i, j, k, such that points2D[i,:], points2D[j,:], points2D[k,:]  are vertices of a triangle in the Delaunay triangulation of the rectangle $U$.


The images  of the `points2D` through the surface parameterization are 3D points. The same simplices define the triangles on the surface.

Setting   a combination of  keys in `Mesh3d` leads to generating and plotting  a tri-surface, in the same way as `plot_trisurf` in matplotlib  or `trisurf` in Matlab does.

We note that `Mesh3d` with different combination of keys can generate  [alpha-shapes](http://nbviewer.jupyter.org/github/empet/Plotly-plots/blob/master/Plotly-Mesh3d.ipynb).

In order to plot a tri-surface, we choose a colormap, and associate to each triangle on the surface,  the  color in colormap, corresponding to  the normalized mean value of z-coordinates of the triangle vertices.

Define a function that maps a mean z-value to a matplotlib color, converted to a Plotly color:

To plot the triangles on a surface,  we set in Plotly `Mesh3d` the lists of x, y, respectively z- coordinates of the vertices, and the lists of indices, i, j, k, for x, y, z coordinates of  all  vertices:

In [65]:
def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value val to a corresponding color in the mpl colormap
    
    if vmin>=vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    C=map(np.uint8, np.array(colormap(t)[:3])*255)
    #convert to a Plotly color code:
    return 'rgb'+str((C[0], C[1], C[2]))
       

Function that converts a matplotlib colormap to a Plotly colorscale with `pl_entries`:

In [67]:
def mpl_to_plotly(cmap, pl_entries):
    h=1.0/(pl_entries-1)
    pl_colorscale=[]
    for k in range(pl_entries):
        C=map(np.uint8, np.array(cmap(k*h)[:3])*255)
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
    return pl_colorscale

Now we define a function that returns data  for a Plotly plot of a tri-surface:

In [68]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly
plotly.offline.init_notebook_mode() 

In [88]:
def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, showcolorbar=False, plot_edges=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangulation;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data
    
    points3D=np.vstack((x,y,z)).T
    
    tri_vertices= points3D[simplices]# vertices of the surface triangles  
   
    zmean=tri_vertices[:, :, 2].mean(-1)# mean values of z-coordinates of the
                                        #triangle vertices
    
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)  
   
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean] 
    I,J,K=zip(*simplices)
    
    triangles=Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor, 
                     i=I,
                     j=J,
                     k=K,
                     name=''
                    )
    if showcolorbar==True:
        pl_colorsc=mpl_to_plotly(colormap,11)
        # a fake Scatter3d defined in order to display the colorbar for the trisurf
        colorbar = Scatter3d(x=tri_vertices[:,0,0],
                             y=tri_vertices[:,0,1],
                             z=tri_vertices[:,0,2],
                             mode='markers',
                             marker=dict(size=0.1, color=zmean, colorscale=pl_colorsc, showscale=True),
                             hoverinfo='None')

    
    
    if plot_edges is None: # the triangle sides are not plotted 
        if  showcolorbar is True:
            return Data([colorbar, triangles])
        else: 
            return  Data([triangles])
    else:#plot edges
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        Xe=[]
        Ye=[]
        Ze=[]
        for T in tri_vertices:
            Xe+=[T[k%3][0] for k in range(4)]+[ None]
            Ye+=[T[k%3][1] for k in range(4)]+[ None]
            Ze+=[T[k%3][2] for k in range(4)]+[ None]
       
        #define the lines to be plotted
        lines=Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=Line(color= 'rgb(50,50,50)', width=1.5)
               )
        if  showcolorbar is True:
            return Data([colorbar, triangles, lines])
        else: 
            
            return Data([triangles, lines])

Call this  function for data associated to Moebius band:

In [70]:
data1=plotly_trisurf(x,y,z, tri.simplices, colormap=cm.RdBu, showcolorbar=True, plot_edges=True)

Set the  layout of the plot:

In [71]:
axis = dict(
showbackground=True, 
backgroundcolor="rgb(230, 230,230)",
gridcolor="rgb(255, 255, 255)",      
zerolinecolor="rgb(255, 255, 255)",  
    )

layout = Layout(
         title='Moebius band triangulation',
         width=800,
         height=800,
         showlegend=False,
         scene=Scene(xaxis=XAxis(axis),
                     yaxis=YAxis(axis), 
                     zaxis=ZAxis(axis), 
                     aspectratio=dict(x=1,
                                      y=1,
                                      z=0.5
                                     ),
                    )
        )

fig1 = Figure(data=data1, layout=layout)

In [72]:

plotly.offline.iplot(fig1)

### Triangulation of the surface $z=\sin(-xy)$, defined over a disk ###

We consider polar coordinates on the disk, $D(0, 1)$, centered at origin and of radius 1, and define
a meshgrid on the set of points $(r, \theta)$, with $r\in[0,1]$ and $\theta\in[0,2\pi]$:

In [73]:
n=12# number of radii
h=1.0/(n-1)
r = np.linspace(h, 1.0, n)
theta= np.linspace(0, 2*np.pi, 36)

r,theta=np.meshgrid(r,theta)
r=r.flatten()
theta=theta.flatten()

#Convert polar coordinates to cartesian coordinates (x,y)
x=r*np.cos(theta)
y=r*np.sin(theta)
x=np.append(x, 0)#  a trick to include the center of the disk in the set of points. It was avoided
                 # initially when we defined r=np.linspace(h, 1.0, n)
y=np.append(y,0)
z = np.sin(-x*y) 

points2D=np.vstack([x,y]).T
tri=Delaunay(points2D)

Plot the  surface with a modified layout:

In [74]:
data2=plotly_trisurf(x,y,z, tri.simplices, colormap=cm.cubehelix, showcolorbar=True, plot_edges=None)
fig2 = Figure(data=data2, layout=layout)
fig2['layout'].update(dict(title='Triangulated surface',
                          scene=dict(camera=dict(eye=dict(x=1.75, 
                                                          y=-0.7, 
                                                          z= 0.75)
                                                )
                                    )))


In [75]:
plotly.offline.iplot(fig2)

### Plotting tri-surfaces from data stored in  ply-files ###

A PLY (Polygon File Format or Stanford Triangle Format) format is a format for storing graphical objects
that are represented by  a triangulation of an object, resulted usually from scanning that object. A Ply file contains the coordinates of vertices, the codes for faces (triangles) and other elements, as well as the color for faces or the normal direction to faces.

In the following we show how we can read a ply file via the Python package, `plyfile`. This package can be installed with `pip`.

In [76]:
from plyfile import PlyData, PlyElement

We choose a ply file from a list  provided [here](http://people.sc.fsu.edu/~jburkardt/data/ply/ply.html).

In [77]:
import urllib2
req = urllib2.Request('http://people.sc.fsu.edu/~jburkardt/data/ply/chopper.ply') 
opener = urllib2.build_opener()
f = opener.open(req)
plydata = PlyData.read(f)

Read the file header:

In [78]:
for element in plydata.elements:
    print element

element vertex 1066
property float x
property float y
property float z
element face 2094
property list uchar int vertex_indices


In [79]:
nr_points=plydata.elements[0].count
nr_faces=plydata.elements[1].count

Read the vertex coordinates:

In [80]:
points=np.array([plydata['vertex'][k] for k in range(nr_points)])
points[0]

(-39.49470138549805, 160.3179931640625, 4.016839981079102)

In [81]:
x,y,z=zip(*points)

In [82]:
faces=[plydata['face'][k][0] for k in range(nr_faces)]
faces[0]

array([0, 1, 2], dtype=int32)

Now we can get data for a Plotly plot of the graphical object read from the ply file:

In [83]:
data3=plotly_trisurf(x,y,z, faces, colormap=cm.RdBu, plot_edges=None)

Update the layout for this new plot:

In [84]:
title="Trisurf from a PLY file<br>"+\
                "Data Source:<a href='http://people.sc.fsu.edu/~jburkardt/data/ply/airplane.ply'> [1]</a>"

In [85]:
noaxis=dict(showbackground=False,
            showline=False,  
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='' 
          )

fig3 = Figure(data=data3, layout=layout)
fig3['layout'].update(dict(title=title,
                           width=1000,
                           height=1000,
                           scene=dict(xaxis=noaxis,
                                      yaxis=noaxis, 
                                      zaxis=noaxis, 
                                      aspectratio=dict(x=1, y=1, z=0.4),
                                      camera=dict(eye=dict(x=1.25, y=1.25, z= 1.25)     
                                     )
                           )
                     ))
                      

In [86]:
plotly.offline.iplot(fig3)

### Plotly plot of an isosurface ###

An isosurface, F(x,y,z) = c, is discretized by a triangular mesh, extracted by the [Marching cubes algorithm](http://scikit-image.org/docs/dev/auto_examples/edges/plot_marching_cubes.html) from a volume given as a (M, N, P) array of doubles. 

The scikit image function, measure.marching_cubes(F, c) returns the vertices and simplices of the triangulated surface.


In [94]:
from skimage import measure
import cmocean
X,Y,Z = np.mgrid[-2:2:40j, -2:2:40j, -2:2:40j]
F = X**4 + Y**4 + Z**4 - (X**2+Y**2+Z**2)**2 + 3*(X**2+Y**2+Z**2) - 3  
vertices, simplices = measure.marching_cubes(F, 0)
x,y,z = zip(*vertices) 

In [97]:
data4=plotly_trisurf(x,y,z, simplices, colormap=cmocean.cm.waveheight_r, showcolorbar=True)
fig4 = Figure(data=data4, layout=layout)
fig4['layout'].update(dict(title='Isosurface',
                          scene=dict(camera=dict(eye=dict(x=1, 
                                                          y=1, 
                                                          z=1)
                                                ),
                                      aspectratio=dict(x=1, y=1, z=1)
                                    )))
plotly.offline.iplot(fig4)


In [98]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()